# Python libraries
- Scientific Computation
  - NumPy
  - SciPy: functions for linear algebra, optimization, integration and statistics (built on top of NumPy)
  - Statsmodels
  - Pandas: "relational" data. Series (one dimension) and Data Frames (two dimensional)
- Data Visualization
  - MatplotLib
  - Seaborn: complimentary to Matplotlib. targets statistical data visualizations
- Machine Learning
  - Scikit-Learn
- Deep Learning (Keras / TensorFlow)
  - TensorFlow: data flow graph computations
  - Keras: neural networks. Can use Theano or TensorFlow

# Pandas

In [9]:
import pandas as pd

df = pd.read_csv('Data/ACS_16_5YR_B24011_with_ann.csv', usecols=['GEO.id', 'GEO.id2'], encoding='latin-1')
df.head(2)

,GEO.id,GEO.id2
0,Id,Id2
1,0500000US01001,01001


In [10]:
# excel sheet names
workbook = pd.ExcelFile('Data/Yelp_Selected_Businesses.xlsx')
workbook.sheet_names

ImportError: Install xlrd >= 1.0.0 for Excel support